# Group-lvl parcellation : from the average of the Similatriy Matrix

In [ ]:
from pathlib import Path
import numpy as np
import nibabel as nib 
import networkx as nx

from parcellation_surface.preprocessing_surface import load_data_normalized
from parcellation_surface.similarity_matrix import compute_similarity_matrix_pca
from parcellation_surface.smoothing import smooth_surface_graph
from parcellation_surface.gradient import compute_gradients, build_mesh_graph
from parcellation_surface.watershed import watershed_by_flooding
from parcellation_surface.visualization import visualize_brain_surface

### Run the cell below for a Gordon like parcellation

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>>>> To modify if needed
dataset = 'PPSFACE_N18'
run = 1
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(f"D:\Data_Conn_Preproc\{dataset}")
}
# Output paths
output_dir = config["subjects_dir"] / "group_parcellation"
(output_dir).mkdir(exist_ok=True, parents=True)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

for hemisphere in ['lh', 'rh']:
    
    surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
    surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
    # Extract the Surface Mesh
    coords, faces = nib.freesurfer.read_geometry(str(surface_path))
    coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
    graph = build_mesh_graph(faces)
    
    # 1 : Compute the average similartiy matrix across each subjects
    sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
    print('sim_matrix_sum shape', sim_matrix_sum.shape)
    for i in range(1,19):
        if i == 5 and dataset == 'PPSFACE_N20':
            continue # Subject 5 is missing PPSFACE20
        
        # >>>>>>>>>>>>>>>>>>> To modify if needed
        # Variables path
        subject = f"{i:02d}"
        subj_dir = config["subjects_dir"] / f"sub-{subject}"

        # Define paths using pathlib
        surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
        vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
        brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"
        # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        
        # Load the data
        surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                       vol_fmri_path, 
                                                       brain_mask_path)
        
        sim_matrix_sum += compute_similarity_matrix_pca(vol_fmri_n,
                                                       surf_fmri_n)

    # Save the similarty matrix
    sim_matrix_path = output_dir / f"similarity_matrix_group_run{run}_{hemisphere}.npy"
    np.save(sim_matrix_path, sim_matrix_sum)
    
    # 2 : Smooth the similatry matrix
    sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

    # 3 : Compute the gradient of the similarty matrix
    gradients = compute_gradients(graph, sim_matrix_smoothed)

    # 3.1 : Smooth the gradient
    gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
    gradients_path = output_dir / f"gradients_smoothed_group_run{run}_{hemisphere}.npy"
    np.save(gradients_path, gradients_smoothed)

    # 4 : Create the boundary map from the watershed
    boundary_map = np.zeros_like(gradients[:,0]).astype(np.float32)
    for map_idx in range(gradients.shape[1]): # loop over each columns of the matrix
        if map_idx % 10 == 0:
            print(f"Processing map {map_idx}") # Print the current map index
        boundary = (watershed_by_flooding(graph, gradients_smoothed[:,map_idx])<0)*1 # Extract the bounary from the watershed algorithm
        boundary_map += boundary
    boundary_map_path = output_dir / f"boundary_map_group_run{run}_{hemisphere}.npy"
    np.save(boundary_map_path, boundary_map)

    # Create the FINAL parcels
    bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
    label_bound = watershed_by_flooding(graph, bound_smoothed)
    # Save the parcels
    np.save(output_dir / f"labels_group_run12_smooth10_run{run}_{hemisphere}.npy", label_bound)

### This Cell is faster but doesn't follow the litterature

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>>>> To modify if needed
dataset = 'PPSFACE_N18'
run = 1
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(f"D:\Data_Conn_Preproc\{dataset}")
}
# Output paths
output_dir = config["subjects_dir"] / "group_parcellation"
(output_dir).mkdir(exist_ok=True, parents=True)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

for hemisphere in ['lh', 'rh']:
    
    surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
    surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"
    # Extract the Surface Mesh
    coords, faces = nib.freesurfer.read_geometry(str(surface_path))
    coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
    graph = build_mesh_graph(faces)
    
    # 1 : Compute the average similartiy matrix across each subjects
    sim_matrix_sum = np.zeros(nx.adjacency_matrix(graph).shape, dtype=np.float32)
    print('sim_matrix_sum shape', sim_matrix_sum.shape)
    for i in range(1,19):
        if i == 5 and dataset == 'PPSFACE_N20':
            continue # Subject 5 is missing PPSFACE20
        
        # >>>>>>>>>>>>>>>>>>> To modify if needed
        # Variables path
        subject = f"{i:02d}"
        subj_dir = config["subjects_dir"] / f"sub-{subject}"

        # Define paths using pathlib
        surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
        vol_fmri_path = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
        brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"
        # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        
        # Load the data
        surf_fmri_n, vol_fmri_n = load_data_normalized(surf_fmri_path,
                                                       vol_fmri_path, 
                                                       brain_mask_path)
        
        sim_matrix_sum += compute_similarity_matrix_pca(vol_fmri_n,
                                                       surf_fmri_n)

    # Save the similarty matrix
    # sim_matrix_path = output_dir / f"similarity_matrix_group_fastway_run{run}_{hemisphere}.npy"
    # np.save(sim_matrix_path, sim_matrix_sum)
    
    # 2 : Smooth the similatry matrix
    sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix_sum, iterations=5)

    # 3 : Compute the gradient of the similarty matrix
    gradients = compute_gradients(graph, sim_matrix_smoothed)

    # 3.1 : Smooth the gradient
    gradients_smoothed = smooth_surface_graph(graph, gradients, iterations=10)
    gradients_path = output_dir / f"gradients_smoothed_group_fastway_run{run}_{hemisphere}.npy"
    np.save(gradients_path, gradients_smoothed)

    # 4 : Create the boundary map from the watershed
    boundary_map = gradients_smoothed.mean(axis=1)

    # Create the FINAL parcels
    bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
    label_bound = watershed_by_flooding(graph, bound_smoothed)
    # Save the parcels
    np.save(output_dir / f"labels_group_fastway_smooth10_run{run}_{hemisphere}.npy", label_bound)

In [ ]:
# WARNING BEFOR USING THIS SCRIPT
"""
This part is used in order to create a parcellation from the run1 and run2.
"""
ppsface = "PPSFACE_N20"
for hemi in ["lh", "rh"]:
   surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
   inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
   # surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
   coords, faces = nib.freesurfer.read_geometry(surface_path)
   coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
   graph = build_mesh_graph(faces)
   bound_sum = 0
   for run in ["1","2"]:
      # Extract the parcellation
      bound_sum += np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
        
   # Create the parcels
   bound_smoothed = smooth_surface_graph(graph, bound_sum, iterations=10)
   label_bound = watershed_by_flooding(graph, bound_smoothed)
   # Save the parcels
   np.save(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_{hemi}.npy", label_bound)

## Show the Results

In [ ]:
# 5 : Visualize the bounardy map
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N20")
}
run = 2
surface_inf_path = config["fsavg6_dir"] / "surf" / f"lh.inflated"
surface_path = config["fsavg6_dir"] / "surf" / f"lh.white"

coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
coords, faces = nib.freesurfer.read_geometry(str(surface_path))
graph = build_mesh_graph(faces)



# path_sim_mtrx = r"D:\Data_Conn_Preproc\PPSFACE_N18\group_parcellation\similarity_matrix_group_run1_lh.npy"
# sim_mtrx = np.load(path_sim_mtrx)
path_grad = r"D:\Data_Conn_Preproc\PPSFACE_N18\group_parcellation\gradients_smoothed_group_run1_lh.npy"
grad = np.load(path_grad)

print('sim_mtrx shape', grad.shape)


In [ ]:
from parcellation_surface.smoothing import smooth_surface_graph

grad_s = smooth_surface_graph(graph, grad[:,10], iterations=10)
label_bound = watershed_by_flooding(graph, grad_s)
visualize_brain_surface(coords_, faces_, (label_bound<0)*1, title='Bounardy Map lh PPSFACE20')

In [ ]:
bound_smoothed = smooth_surface_graph(graph, boundary_map, iterations=10)
visualize_brain_surface(coords_, faces_, bound_smoothed)

In [ ]:
labels = watershed_by_flooding(graph, bound_smoothed)
visualize_brain_surface(coords_, faces_, labels, title="Group-Level Parcellation lh PPSFACE20")